1- Seleccionar un target del data set

2- Que problema puedo solucionar con esa columna ? (regresion o clasificacions )

3- Comenzar con la preparacion del data set (x,y) para lebels y features

4-Eliminar datos vacios o rellenar

5- Dummy hotencoding (transform data to numeric)

6- x todas las features y la y son las lebels que se toman de la columna que escogi

In [1]:
import pandas as pd

# Ruta al archivo local
file_path = '/content/Indicadores_municipales_sabana_DA.csv'

# Cargar el conjunto de datos en un DataFrame de pandas
df = pd.read_csv(file_path, encoding='latin-1')

# Mostrar las primeras filas del DataFrame para verificar que se haya cargado correctamente
df.head()

,ent,nom_ent,mun,clave_mun,nom_mun,pobtot_ajustada,pobreza,pobreza_e,pobreza_m,vul_car,...,pobreza_alim_10,pobreza_cap_90,pobreza_cap_00,pobreza_cap_10,pobreza_patrim_90,pobreza_patrim_00,pobreza_patrim_10,gini_90,gini_00,gini_10
0,1,Aguascalientes,1,1001,Aguascalientes,794304,30.531104,2.264478,28.266627,27.983320,...,11.805700,20.4,12.7,18.474600,43.4,33.7,41.900398,0.473,0.425,0.422628
1,1,Aguascalientes,2,1002,Asientos,48592,67.111172,8.040704,59.070468,22.439389,...,21.993299,39.9,29.0,30.980801,64.2,48.9,59.175800,0.379,0.533,0.343879
2,1,Aguascalientes,3,1003,Calvillo,53104,61.360527,7.241238,54.119289,29.428583,...,19.266800,39.5,33.1,28.259199,63.9,57.9,56.504902,0.414,0.465,0.386781
3,1,Aguascalientes,4,1004,Cosío,14101,52.800458,4.769001,48.031458,27.128568,...,14.303200,35.2,21.0,22.386101,59.7,40.1,51.164501,0.392,0.541,0.344984
4,1,Aguascalientes,5,1005,Jesús María,101379,45.338512,6.084037,39.254475,26.262912,...,15.085100,36.6,22.6,22.139999,60.6,42.2,45.703899,0.391,0.469,0.458083


Divide my features into 'y' and 'x'

In [2]:
columns_to_exclude = ["ent", "nom_ent", "mun","nom_mun","N_ic_rezedu"]
X = df.drop(columns_to_exclude, axis=1)

y = df["N_ic_rezedu"]

print("data already divided")
print(X.columns)


data already divided
Index(['clave_mun', 'pobtot_ajustada', 'pobreza', 'pobreza_e', 'pobreza_m',
       'vul_car', 'vul_ing', 'npnv', 'ic_rezedu', 'ic_asalud',
       ...
       'pobreza_alim_10', 'pobreza_cap_90', 'pobreza_cap_00', 'pobreza_cap_10',
       'pobreza_patrim_90', 'pobreza_patrim_00', 'pobreza_patrim_10',
       'gini_90', 'gini_00', 'gini_10'],
      dtype='object', length=134)


We codify the columns with string values and filled the Nan values in the dataset

In [3]:
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.impute import SimpleImputer

label_encoder = LabelEncoder()
X['gdo_rezsoc00'] = label_encoder.fit_transform(X['gdo_rezsoc00'])
X['gdo_rezsoc05'] = label_encoder.fit_transform(X['gdo_rezsoc05'])
X['gdo_rezsoc10'] = label_encoder.fit_transform(X['gdo_rezsoc10'])

imputer = SimpleImputer(strategy="mean")

X_imputed = imputer.fit_transform(X)


scaler = StandardScaler()
X_scaler = scaler.fit_transform(X_imputed)

The data is divided into 80% for train an 20% for test.

In [4]:
from sklearn.model_selection import train_test_split


X_train, X_test, y_train, y_test = train_test_split(X_scaler, y, test_size=0.2, random_state=42)



WE import a model for the regression solution

In [5]:
from sklearn.linear_model import LinearRegression

modelo_regresion = LinearRegression()


The model it is trained

In [6]:
modelo_regresion.fit(X_train, y_train)

LinearRegression()

The model predict the number of people in poverty

In [7]:
y_pred = modelo_regresion.predict(X_test)

We show the prediction and the corresponding municipality and it is compared with the real value in the csv.

In [8]:
X_test_df = pd.DataFrame(X_test, columns=X.columns)
codigos_municipio = X_test_df['clave_mun']



if len(y_pred) == len(y_test):
    for i in range(len(y_pred)):
        codigo_municipio = codigos_municipio.iloc[i]
        prediccion_pobreza = y_pred[i]
        pobreza_actual = y_test.iloc[i]
        print(f"Para el municipio con el código {codigo_municipio}, se predice que habrá {prediccion_pobreza} personas con rezago educativo. El valor real de 'N_ic_rezedu' es {pobreza_actual}.")
else:
    print("Las dimensiones de y_pred y y_test no coinciden.")

Para el municipio con el código 0.12838562200511897, se predice que habrá 2802.4240320780837 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 3045.
Para el municipio con el código -0.7168338294212445, se predice que habrá 11518.155567178466 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 11963.
Para el municipio con el código 0.6354357735439694, se predice que habrá 1142.0666486162572 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 2390.
Para el municipio con el código -0.7128937291011409, se predice que habrá 149867.73460615403 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 163920.
Para el municipio con el código -0.5792020492741772, se predice que habrá 109787.84784597302 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 112444.
Para el municipio con el código 0.151211030756064, se predice que habrá 1704.4664143340642 personas con rezago educativo. El valor real de 'N_ic_rezedu' es 363.
Para el municip

We assess the model

In [9]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error (MSE): {mse}")
print(f"R^2: {r2}")


Mean Squared Error (MSE): 7063990.982293092
R^2: 0.9841521152540307
